In [21]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 1000
learning_rate = 1e-3

img_transform = transforms.Compose([
transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

from PIL import  Image,ImageDraw
import numpy as np


data1 = []
data2= []

for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')

    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')

    draw = ImageDraw.Draw(im2)

    draw.rectangle([21,12,24,15],fill='red',outline='red')
    data2.append(np.asarray(im2))


for i in range(20000):
    img = Image.new('RGB', (28, 28), color = 'white')


    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')

    draw = ImageDraw.Draw(im2)

    draw.rectangle([9,12,12,15],fill='red',outline='red')
    data1.append(np.asarray(im2))
    

y0 = np.array(data1, dtype=np.float32)





y1 = np.array(data2, dtype=np.float32)
y0 = np.transpose(y0, (0, 3, 2, 1))# This to make channels first for CNN
y1 = np.transpose(y1, (0, 3, 2, 1)) 


print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)



torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=250,

    shuffle=True,

    num_workers=0,

)

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True), # b, 8, 2, 2
            nn.Conv2d(8,4,3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(4,2,3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
        )
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = x.view(-1,2)
        x = self.fc(x)
        return x

class auto_decoder(nn.Module):
    def __init__(self):
        super(auto_decoder,self).__init__()
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 3,padding=1)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 3,padding=1)
        
    def forward(self, x):
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x
    

model = autoencoder()


def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

model_1 = auto_decoder()
optimizer_1 = torch.optim.Adam(model_1.parameters(), lr=learning_rate,
                             weight_decay=1e-5)


for epoch in range(num_epochs):
    for x,y in loader:

        # ===================forward=====================
        #loss = criterion(output1, img)
        loss = h_score(model(x),model(y))
        # ===================backward====================
        optimizer.zero_grad()
        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.item()))
    print("x: ", model(x[0].reshape(1,3,28,28)))
    print("y:", model(y[0].reshape(1,3,28,28)))
    print("b:", model(b.reshape(1,3,28,28)))


torch.save(model.state_dict(), './conv_autoencoder.pth')

(20000, 3, 28, 28)
epoch [1/100], loss:0.0039
x:  tensor([[-1.1725]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6766]], grad_fn=<AddmmBackward>)
b: tensor([[-1.2536]], grad_fn=<AddmmBackward>)
epoch [2/100], loss:-0.0003
x:  tensor([[-1.5316]], grad_fn=<AddmmBackward>)
y: tensor([[-1.5496]], grad_fn=<AddmmBackward>)
b: tensor([[-1.2849]], grad_fn=<AddmmBackward>)
epoch [3/100], loss:0.0001
x:  tensor([[-1.7062]], grad_fn=<AddmmBackward>)
y: tensor([[-1.7218]], grad_fn=<AddmmBackward>)
b: tensor([[-1.3254]], grad_fn=<AddmmBackward>)
epoch [4/100], loss:-0.0049
x:  tensor([[-1.4099]], grad_fn=<AddmmBackward>)
y: tensor([[-1.4095]], grad_fn=<AddmmBackward>)
b: tensor([[-1.8360]], grad_fn=<AddmmBackward>)
epoch [5/100], loss:-0.0006
x:  tensor([[-2.6778]], grad_fn=<AddmmBackward>)
y: tensor([[-2.9928]], grad_fn=<AddmmBackward>)
b: tensor([[-2.3238]], grad_fn=<AddmmBackward>)
epoch [6/100], loss:0.0024
x:  tensor([[-1.9077]], grad_fn=<AddmmBackward>)
y: tensor([[-2.2913]], grad_fn=<AddmmBackw

epoch [49/100], loss:-0.0750
x:  tensor([[-2.1694]], grad_fn=<AddmmBackward>)
y: tensor([[-2.3300]], grad_fn=<AddmmBackward>)
b: tensor([[-2.4101]], grad_fn=<AddmmBackward>)
epoch [50/100], loss:-0.0530
x:  tensor([[-3.2212]], grad_fn=<AddmmBackward>)
y: tensor([[-2.8714]], grad_fn=<AddmmBackward>)
b: tensor([[-3.0161]], grad_fn=<AddmmBackward>)
epoch [51/100], loss:-0.0534
x:  tensor([[-3.2445]], grad_fn=<AddmmBackward>)
y: tensor([[-2.5536]], grad_fn=<AddmmBackward>)
b: tensor([[-2.8618]], grad_fn=<AddmmBackward>)
epoch [52/100], loss:-0.0830
x:  tensor([[-2.2289]], grad_fn=<AddmmBackward>)
y: tensor([[-2.2903]], grad_fn=<AddmmBackward>)
b: tensor([[-2.3270]], grad_fn=<AddmmBackward>)
epoch [53/100], loss:-0.0582
x:  tensor([[-2.4165]], grad_fn=<AddmmBackward>)
y: tensor([[-2.3332]], grad_fn=<AddmmBackward>)
b: tensor([[-2.0673]], grad_fn=<AddmmBackward>)
epoch [54/100], loss:-0.0407
x:  tensor([[-2.2359]], grad_fn=<AddmmBackward>)
y: tensor([[-2.7406]], grad_fn=<AddmmBackward>)
b: t

epoch [97/100], loss:-0.1222
x:  tensor([[-3.0866]], grad_fn=<AddmmBackward>)
y: tensor([[-2.7153]], grad_fn=<AddmmBackward>)
b: tensor([[-2.7683]], grad_fn=<AddmmBackward>)
epoch [98/100], loss:-0.0790
x:  tensor([[-2.0000]], grad_fn=<AddmmBackward>)
y: tensor([[-2.8358]], grad_fn=<AddmmBackward>)
b: tensor([[-2.7517]], grad_fn=<AddmmBackward>)
epoch [99/100], loss:-0.0426
x:  tensor([[-2.7606]], grad_fn=<AddmmBackward>)
y: tensor([[-1.9728]], grad_fn=<AddmmBackward>)
b: tensor([[-3.0697]], grad_fn=<AddmmBackward>)
epoch [100/100], loss:-0.0520
x:  tensor([[-1.2705]], grad_fn=<AddmmBackward>)
y: tensor([[-1.7560]], grad_fn=<AddmmBackward>)
b: tensor([[-2.5144]], grad_fn=<AddmmBackward>)


In [15]:
blank = cv2.imread('blank_28x28.png')
b = blank.astype('float32')
b = torch.from_numpy(b)

for i in range(5,15):
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))

print("b:", model(b.reshape(1,3,28,28)))


x:  tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[18.8033]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[25.8582]],

         [[16.1655]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
x:  tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[33.6211]],

         [[28.7243]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
y: tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[27.2459]],

         [[28.3437]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]],

         [[ 0.0000]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
x:  tensor([[[[ 0.0000]],

         [[ 0.0000]],

         [[36.0313]],

         [[17

In [22]:
for i in range(10):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2 = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2)
    b = np.array(b, dtype=np.float32)
    b = torch.from_numpy(b)
    print("b:", model(b.reshape(1,3,28,28)))

for i in range(5,15):
    print("x: ", model(x[i].reshape(1,3,28,28)))
    print("y:", model(y[i].reshape(1,3,28,28)))

b: tensor([[-2.7583]], grad_fn=<AddmmBackward>)
b: tensor([[-1.8192]], grad_fn=<AddmmBackward>)
b: tensor([[-2.1667]], grad_fn=<AddmmBackward>)
b: tensor([[-2.3053]], grad_fn=<AddmmBackward>)
b: tensor([[-1.8137]], grad_fn=<AddmmBackward>)
b: tensor([[-2.9625]], grad_fn=<AddmmBackward>)
b: tensor([[-1.4496]], grad_fn=<AddmmBackward>)
b: tensor([[-2.1995]], grad_fn=<AddmmBackward>)
b: tensor([[-2.6051]], grad_fn=<AddmmBackward>)
b: tensor([[-2.7833]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.0748]], grad_fn=<AddmmBackward>)
y: tensor([[-2.7647]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.1197]], grad_fn=<AddmmBackward>)
y: tensor([[-2.2255]], grad_fn=<AddmmBackward>)
x:  tensor([[-3.1761]], grad_fn=<AddmmBackward>)
y: tensor([[-2.6508]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.5279]], grad_fn=<AddmmBackward>)
y: tensor([[-1.6658]], grad_fn=<AddmmBackward>)
x:  tensor([[-1.9616]], grad_fn=<AddmmBackward>)
y: tensor([[-2.9086]], grad_fn=<AddmmBackward>)
x:  tensor([[-2.5519]], grad_fn=<Ad